## Generate parameter set for graph optimizations

In [44]:
from copy import deepcopy
from random import choice

# generate some non-positional optimizations
number_of_non_positional_optimizations = 7
np_opts = []
for i in range(number_of_non_positional_optimizations):
    np_opt = {
        str(i): {
            "positions": 1,
            "diagonal": choice([-5, 5]),
            "nodes_with_edges": choice([-5, 5, 10, None]),
            "edges": choice([-5, 5, 10, None]),
            "non_edges": choice([-5, 5, 10, None])
        }
    }

    # remove any keys where None was selected
    np_opts.append(
        {k: {k2: value for k2, value in np_opt.get(k).items() if value is not None} for
         k, _ in np_opt.items()}
    )

# generate some positional optimizations
p_opts = []
number_of_positional_optimizations = 6  # this will be doubled, once for pos=colors, once for pos=n
for i in range(number_of_positional_optimizations):
    p_opt_c = {
        str(i): {
            "positions": "colors",
            "start_node_score": choice([-5, 5, None]),
            "terminal_node_score": choice([-5, 5, None]),
            "diagonal": choice([-5, 5, -10]),
            "one_node_many_positions": choice([5, 10, None]),
            "one_position_many_nodes": choice([5, 10, None]),
            "edges": choice([5, 10, None]),
            "edge_weights_factor": choice([-1, 1, None]),
            "edge_weights_cycles_factor": choice([-1, 1, None]),
            "edge_weights_self_factor": choice([-1, 1, None]),
            "non_edges": choice([5, 10, None]),
            "invalid_traversal": choice([5, None]),
            "invalid_traversal_cycle": choice([5, None]),
            "invalid_traversal_self": choice([5, None])
        }
    }
    p_opt_n = deepcopy(p_opt_c)
    p_opt_n[str(i)]["positions"] = "n"

    p_opts.append(
        {k: {k2: value for k2, value in p_opt_c.get(k).items() if value is not None} for
         k, _ in p_opt_c.items()}
    )
    p_opts.append(
        {k: {k2: value for k2, value in p_opt_n.get(k).items() if value is not None} for
         k, _ in p_opt_n.items()}
    )
# These are saved manually to graph_optimizations.json
np_opts + p_opts

[{'0': {'positions': 1, 'diagonal': 5, 'non_edges': -5}},
 {'1': {'positions': 1,
   'diagonal': -5,
   'nodes_with_edges': -5,
   'edges': 10,
   'non_edges': -5}},
 {'2': {'positions': 1, 'diagonal': 5}},
 {'3': {'positions': 1,
   'diagonal': 5,
   'nodes_with_edges': 10,
   'edges': 10,
   'non_edges': 10}},
 {'4': {'positions': 1, 'diagonal': -5, 'nodes_with_edges': 10}},
 {'5': {'positions': 1, 'diagonal': 5, 'nodes_with_edges': -5, 'edges': -5}},
 {'6': {'positions': 1,
   'diagonal': -5,
   'nodes_with_edges': 5,
   'non_edges': -5}},
 {'0': {'positions': 'colors',
   'start_node_score': -5,
   'terminal_node_score': -5,
   'diagonal': -10,
   'one_node_many_positions': 5,
   'one_position_many_nodes': 10,
   'edges': 5,
   'edge_weights_factor': 1,
   'edge_weights_cycles_factor': -1,
   'edge_weights_self_factor': 1}},
 {'0': {'positions': 'n',
   'start_node_score': -5,
   'terminal_node_score': -5,
   'diagonal': -10,
   'one_node_many_positions': 5,
   'one_position_many_n

## Generate random graphs for experimentation

In [46]:
from random import randrange
from typing import List

from networkx import Graph

graphs: List[Graph] = []

graph_sizes = [4, 5, 6, 7, 8, 9, 10, 12, 14, 16, 18, 20]
num_graph_per_size = 42

for size in graph_sizes:
    for i in range(num_graph_per_size):
        graph = Graph()
        num_edges = randrange(2, size * (
                size - 2))  # rand in range of max number of possible edges
        edges = []

        for _ in range(num_edges):  # add random number of edges

            u = randrange(size - 1)
            v = randrange(size - 1)
            w = randrange(1, 4)

            if (u, v) not in [(u1, v1) for (u1, v1, _) in edges]:
                edges.append((u, v, w))

        for node in range(size):  # add an edge for any nodes not connected in the graph
            if node not in [u for (u, _, _) in edges] and node not in [v for (_, v, _)
                                                                       in edges]:
                u = node
                v = randrange(size - 1)
                w = randrange(1, 4)

                edges.append((u, v, w))

        graph.add_weighted_edges_from(edges)
        graphs.append(graph)

len(graphs)

504

## Save graphs for repeatability

In [47]:
import json
from pathlib import Path

from networkx import to_dict_of_dicts

with open(Path("graphs.json"), "w") as file:
    json.dump(
        {key: to_dict_of_dicts(value) for key, value in enumerate(graphs)}, file
    )

# Execute classical solving for graphs

In [50]:
from copy import deepcopy
import json
from pathlib import Path
from bachelorthesis.graph_optimization import GraphOptimization

from networkx import from_dict_of_dicts

from bachelorthesis.solvers.qb_solv import solve as solve_qbs
from bachelorthesis.solvers.repeated_simulated_annealing import solve as solve_rsa
from bachelorthesis.solvers.simulated_annealing import solve as solve_sa
from bachelorthesis.solvers.tabu_search import solve as solve_tbs

with open(Path("graphs.json"), "r") as graphs_file:
    saved_graphs = json.load(graphs_file)

with open(Path("graph_optimizations.json"), "r") as g_opts_file:
    optimization_parameters = json.load(g_opts_file)

for graph_id, graph in saved_graphs.items():
    # dict_of_dicts turns int keys into strings, this converts them back to ints
    graph_data = {int(key): {int(key2): value for key2, value in graph.get(key).items()}
                  for key, _ in graph.items()}
    graph = from_dict_of_dicts(graph_data)

    # create graph optimization
    g_opt = GraphOptimization(graph=graph)
    start_node = 0
    terminal_node = graph.order() - 1

    results = []

    for params_id, params in optimization_parameters.items():
        # allow positions in the json to take on a value of "n" for faster problem creation
        params_copy = deepcopy(params)
        pos = params_copy.pop("positions")
        if pos == "n":
            pos = graph.order()
        elif pos == "colors":
            # TODO: this is completely arbitrary, does it make sense??
            pos = graph.order() // 3

        # generate qubo
        qubo = g_opt.generate_qubo(
            positions=pos,
            start_node=start_node,
            terminal_node=terminal_node,
            **params_copy
        )

        # solve qubo and store result with all the relevant solvers

        solution_qbs = solve_qbs(qubo)
        solution_rsa = solve_rsa(qubo, num_repeats=10)
        solution_sa = solve_sa(qubo)
        solution_tbs = solve_tbs(qubo)

        result_entry = {
            "graph_id": graph_id,
            "params_id": params_id,
            "qubo": qubo.tolist(),
            "solution_qbs_sample": solution_qbs[0][0].tolist(),
            "solution_qbs_energy": solution_qbs[1].tolist()[0],
            "solution_rsa_sample": solution_rsa[0][0].tolist(),
            "solution_rsa_energy": solution_rsa[1].tolist()[0],
            "solution_sa_sample": solution_sa[0][0].tolist(),
            "solution_sa_energy": solution_sa[1].tolist()[0],
            "solution_tbs_sample": solution_tbs[0][0].tolist(),
            "solution_tbs_energy": solution_tbs[1].tolist()[0]
        }
        results.append(result_entry)

    # store results per graph because i/o is relatively expensive
    with open(Path("results.json"), "r") as results_file_in:
        current_results = json.load(results_file_in)
        current_results.extend(results)
    with open(Path("results.json"), "w") as results_file_out:
        json.dump(current_results, results_file_out)


## Execute quantum solving for graphs

In [2]:
import json
from pathlib import Path

from bachelorthesis.solvers.quantum_annealer import solve as solve_quantum, \
    find_embedding

with open(Path("results.json"), "r") as results_file:
    classical_results = json.load(results_file)

best_graph = 46
best_params = 1

for classical_result in classical_results:

    graph = classical_result.get("graph_id")
    params = classical_result.get("params_id")

    if (int(graph) == best_graph and int(params) > best_params) or (int(graph) > best_graph):
        qubo = classical_result.get("qubo")

        # convert list of list qubo format to dictionary for uqo service
        qubo_dict = {}
        for i, inner_list in enumerate(qubo):
            for j, value in enumerate(inner_list):
                qubo_dict[(i, j)] = value

        # if no embedding is found, store that information, otherwise, solve and store
        if not find_embedding(qubo_dict):
            quantum_result_entry = {
                "graph_id": classical_result.get("graph_id"),
                "params_id": classical_result.get("params_id"),
                "qubo_size": len(qubo),
                "solution_sample": "DNE",
                "energy": "DNE"
            }
        else:
            sample, energy = solve_quantum(qubo_dict, repeats=1)

            quantum_result_entry = {
                "graph_id": classical_result.get("graph_id"),
                "params_id": classical_result.get("params_id"),
                "qubo_size": len(qubo),
                "solution_sample": sample,
                "energy": int(energy)
            }

        # store results per graph because i/o is relatively expensive
        with open(Path("results_quantum.json"), "r") as results_file_in:
            current_results = json.load(results_file_in)
            current_results.append(quantum_result_entry)
        with open(Path("results_quantum.json"), "w") as results_file_out:
            json.dump(current_results, results_file_out)

KeyboardInterrupt: 

In [8]:
import datetime
import json
from pathlib import Path

from bachelorthesis.solvers.quantum_annealer import solve as solve_quantum, \
    find_embedding

with open(Path("results.json"), "r") as results_file:
    classical_results = json.load(results_file)

best_graph = 43
best_params = 24

result_batch = []
for i, classical_result in enumerate(classical_results):

    graph = classical_result.get("graph_id")
    params = classical_result.get("params_id")


    if (int(graph) == best_graph and int(params) > best_params) or (int(graph) > best_graph):
        print(f"IDX: {graph}, {params}")

        qubo = classical_result.get("qubo")
        qubo_size = len(qubo)

        # convert list of list qubo format to dictionary for uqo service
        qubo_dict = {}
        for k, inner_list in enumerate(qubo):
            for j, value in enumerate(inner_list):
                qubo_dict[(k, j)] = value

        print(f"QSZ: {qubo_size}")
        before = datetime.datetime.now()

        # if no embedding is found, store that information, otherwise, solve and store
        if not find_embedding(qubo_dict):
            print(f"EMB: failed")
            quantum_result_entry = {
                "graph_id": classical_result.get("graph_id"),
                "params_id": classical_result.get("params_id"),
                "qubo_size": qubo_size,
                "solution_sample": "DNE",
                "energy": "DNE"
            }
        else:
            after = datetime.datetime.now()
            duration = after - before
            duration = divmod(duration.total_seconds(), 60)
            print(f"EMB: success, took {duration[0]} minutes, {duration[1]} seconds")
            sample, energy = solve_quantum(qubo_dict, repeats=10)

            quantum_result_entry = {
                "graph_id": classical_result.get("graph_id"),
                "params_id": classical_result.get("params_id"),
                "qubo_size": qubo_size,
                "solution_sample": sample,
                "energy": int(energy)
            }

            print(f"SLV: complete")

        if i % 20 != 0:
            result_batch.append(quantum_result_entry)
        else:
            result_batch.append(quantum_result_entry)
            # store results per graph because i/o is relatively expensive
            with open(Path("results_quantum.json"), "r") as results_file_in:
                current_results = json.load(results_file_in)
                current_results.extend(result_batch)
            with open(Path("results_quantum.json"), "w") as results_file_out:
                json.dump(current_results, results_file_out)
            result_batch.clear()

IDX: 43, 25
QSZ: 25
EMB: success, took 0.0 minutes, 4.695299 seconds


KeyboardInterrupt: 